In [1]:
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import astropy
import speclite
from astropy.cosmology import default_cosmology
from astropy.units import Quantity
from slsim.lens_pop import LensPop
from slsim.Observations.roman_speclite import configure_roman_filters, filter_names
from tqdm import tqdm
from pprint import pprint
import time
import pandas as pd
import datetime

import mejiro
from mejiro.helpers import survey_sim
from mejiro.utils import util

%matplotlib inline

In [2]:
from hydra import initialize, compose

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

In [3]:
# !jupyter nbconvert --to python hlwas_simulation.ipynb --TemplateExporter.exclude_markdown=True --TemplateExporter.exclude_output_prompt=True --TemplateExporter.exclude_input_prompt=True

In [4]:
start = time.time()

In [5]:
output_dir = os.path.join(data_dir, 'output', 'hlwas_sim')
util.create_directory_if_not_exists(output_dir)
util.clear_directory(output_dir)

In [6]:
area_hlwas = 1700.
cosmo = default_cosmology.get()

bands_hlwas = ['F106', 'F129', 'F184']

survey_area = 0.5
sky_area = Quantity(value=survey_area, unit='deg2')

In [7]:
# load Roman WFI filters
configure_roman_filters()
roman_filters = filter_names()
roman_filters.sort()
_ = speclite.filters.load_filters(*roman_filters[:8])

In [8]:
module_path = os.path.dirname(mejiro.__file__)
skypy_config = os.path.join(module_path, 'data', 'roman_hlwas.yml')

# define cuts on the intrinsic deflector and source populations (in addition to the skypy config file)
kwargs_deflector_cut = {'band': 'F106', 'band_max': 23, 'z_min': 0.01, 'z_max': 2.}
kwargs_source_cut = {'band': 'F106', 'band_max': 24, 'z_min': 0.01, 'z_max': 5.}

# create the lens population
lens_pop = LensPop(deflector_type="all-galaxies",  # elliptical, all-galaxies
    source_type="galaxies",
    kwargs_deflector_cut=kwargs_deflector_cut,
    kwargs_source_cut=kwargs_source_cut,
    kwargs_mass2light=None,
    skypy_config=skypy_config,
    sky_area=sky_area,
    cosmo=cosmo)

In [9]:
num_lenses = lens_pop.deflector_number()
num_sources = lens_pop.source_number()

print(f'Number of deflectors: {num_lenses}, scaled to HLWAS ({area_hlwas} sq deg): {int((area_hlwas / survey_area) * num_lenses)}')
print(f'Number of sources: {num_sources}, scaled to HLWAS ({area_hlwas} sq deg): {int((area_hlwas / survey_area) * num_sources)}')

Number of deflectors: 29449, scaled to HLWAS (1700.0 sq deg): 100126600
Number of sources: 1161413, scaled to HLWAS (1700.0 sq deg): 3948804200


Need to grab parameters for the entire lens population

In [10]:
total_lens_population = lens_pop.draw_population(kwargs_lens_cuts={})
print(f'Number of total lenses: {len(total_lens_population)}')

Number of total lenses: 3893


In [11]:
# compute SNRs and save
snr_list = []
for candidate in tqdm(total_lens_population):
    snr, _ = survey_sim.get_snr(candidate, 'F106')
    snr_list.append(snr)
np.save(os.path.join(output_dir, 'snr_list.npy'), snr_list)

# save other params to CSV
total_pop_csv = os.path.join(output_dir, 'total_pop.csv')
# survey_sim.write_lens_pop_to_csv(total_pop_csv, total_lens_population, bands_hlwas)

  0%|          | 12/3893 [00:00<00:32, 117.82it/s]

100%|██████████| 3893/3893 [00:32<00:00, 120.74it/s]


In [15]:
output_path = total_pop_csv
gg_lenses = total_lens_population
bands = bands_hlwas

dictparaggln = {}
dictparaggln['Candidate'] = {}
listnamepara = ['velodisp', 'massstel', 'angleins', 'redssour', 'redslens', 'xposlens', 'yposlens', 'xpossour',
                'ypossour', 'numbimag', 'magnsour', 'maxmdistimag']
for nameband in bands:
    listnamepara += ['magtlens%s' % nameband]
    listnamepara += ['magtsour%s' % nameband]
    listnamepara += ['magtsourMagnified%s' % nameband]

for namepara in listnamepara:
    dictparaggln['Candidate'][namepara] = np.empty(len(gg_lenses))

df = pd.DataFrame(columns=listnamepara)

for i, gg_lens in tqdm(enumerate(gg_lenses), total=len(gg_lenses)):
    dict = {
        'velodisp': gg_lens.deflector_velocity_dispersion,
        'massstel': gg_lens.deflector_stellar_mass() * 1e-12, 
        'angleins': gg_lens.einstein_radius, 
        'redssour': gg_lens.source_redshift, 
        'redslens': gg_lens.deflector_redshift, 
        'magnsour': gg_lens.extended_source_magnification
        # TODO add SNR?
    }

    posiimag = gg_lens.point_source_image_positions()
    dict['numbimag'] = int(posiimag[0].size)

    dict['maxmdistimag'] = np.amax(np.sqrt((posiimag[0][:, None] - posiimag[0][None, :]) ** 2 + (posiimag[1][:, None] - posiimag[1][None, :]) ** 2))

    posilens = gg_lens.deflector_position
    # posisour = gg_lens.position_alignment()
    dict['xposlens'] = posilens[0]
    dict['yposlens'] = posilens[1]
    dict['xpossour'] = gg_lens._center_source[0]
    dict['ypossour'] = gg_lens._center_source[1]  # TODO otherwise, set with extended_source_position()

    for nameband in bands:
        dict['magtsour%s' % nameband] = gg_lens.source_magnitude(band=nameband)
        dict['magtsourMagnified%s' % nameband] = gg_lens.source_magnitude(band=nameband, lensed=True)
        dict['magtlens%s' % nameband] = gg_lens.deflector_magnitude(band=nameband)

    df.loc[i] = pd.Series(dict)

print('Writing to %s..' % output_path)
if os.path.exists(output_path): 
    os.remove(output_path)
df.to_csv(output_path, index=False)

  0%|          | 0/3893 [00:00<?, ?it/s]


AttributeError: 'Lens' object has no attribute '_center_source'

Now, start to filter down to the detectable ones

In [ ]:
kwargs_lens_cut = {
    'min_image_separation': 0.2,
    'max_image_separation': 5,
    'mag_arc_limit': {'F106': 25},
}

lens_population = lens_pop.draw_population(kwargs_lens_cuts=kwargs_lens_cut)
print(f'Number of valid lenses: {len(lens_population)}')

NB in the current version of `slsim`, `slsim.LensPop.draw_population` is using the method `slsim.lens.validity_test()` to filter out non-detectable lenses. Here, we want to add a few more filters:

1. is the Einstein radius greater than the Sersic radius of the lensing galaxy and the lensing galaxy brighter than 15th magnitude (in F106)?
2. is the SNR of lensed images greater than 10?

In [ ]:
# set up dict to capture some information about which candidates got filtered out
filtered_sample = {}
filtered_sample['total'] = len(lens_population)
num_samples = 100
filter_1, filter_2 = 0, 0
filtered_sample['filter_1'] = []
filtered_sample['filter_2'] = []

In [ ]:
limit = None
detectable_gglenses = []

for candidate in tqdm(lens_population):
    # 1. Einstein radius and Sersic radius
    _, kwargs_params = candidate.lenstronomy_kwargs(band='F106')
    lens_mag = candidate.deflector_magnitude(band='F106')

    if kwargs_params['kwargs_lens'][0]['theta_E'] < kwargs_params['kwargs_lens_light'][0]['R_sersic'] and lens_mag < 15:
        filter_1 += 1
        if filter_1 <= num_samples:
            filtered_sample['filter_1'].append(candidate)
        continue

    # 2. SNR
    snr, _ = survey_sim.get_snr(candidate, 'F106')

    if snr < 10:
        filter_2 += 1
        if filter_2 <= num_samples:
            filtered_sample['filter_2'].append(candidate)
        continue

    # if both criteria satisfied, consider detectable
    detectable_gglenses.append(candidate)

    # if I've imposed a limit above this loop, exit the loop
    if limit is not None and len(detectable_gglenses) == limit:
        break

filtered_sample['num_filter_1'] = filter_1
filtered_sample['num_filter_2'] = filter_2

print(f'{len(detectable_gglenses)} detectable lens(es)')

In [ ]:
if len(detectable_gglenses) > 0:
    pprint(filtered_sample['num_filter_1'], filtered_sample['num_filter_2'])

Now that we have our list of detectable lenses, pickle them for the image simulation part of the pipeline.

In [ ]:
print('Retrieving lenstronomy parameters')
dict_list = []
for gglens in tqdm(detectable_gglenses):

    # get lens params from gglens object
    kwargs_model, kwargs_params = gglens.lenstronomy_kwargs(band='F106')

    # build dicts for lens and source magnitudes
    lens_mags, source_mags = {}, {}
    for band in ['F106', 'F129', 'F184']:
        lens_mags[band] = gglens.deflector_magnitude(band)
        source_mags[band] = gglens.source_magnitude(band)

    z_lens, z_source = gglens.lens_redshift, gglens.source_redshift
    kwargs_lens = kwargs_params['kwargs_lens']

    # add additional necessary key/value pairs to kwargs_model
    kwargs_model['lens_redshift_list'] = [z_lens] * len(kwargs_lens)
    kwargs_model['source_redshift_list'] = [z_source]
    kwargs_model['cosmo'] = cosmo
    kwargs_model['z_source'] = z_source
    kwargs_model['z_source_convention'] = 5

    # create dict to pickle
    gglens_dict = {
        'kwargs_model': kwargs_model,
        'kwargs_params': kwargs_params,
        'lens_mags': lens_mags,
        'source_mags': source_mags,
        'deflector_stellar_mass': gglens.deflector_stellar_mass(),
        'deflector_velocity_dispersion': gglens.deflector_velocity_dispersion(),
        'snr': gglens.snr
    }

    dict_list.append(gglens_dict)

print('Pickling lenses...')
for i, each in tqdm(enumerate(dict_list)):
    save_path = os.path.join(data_dir, 'skypy', f'skypy_output_{str(i).zfill(5)}.pkl')
    util.pickle(save_path, each)

In [ ]:
detectable_pop_csv = os.path.join(output_dir, 'detectable_pop.csv')
survey_sim.write_lens_pop_to_csv(detectable_pop_csv, detectable_gglenses, bands_hlwas)

In [ ]:
stop = time.time()
execution_time = str(datetime.timedelta(seconds=round(stop - start)))
print(f'Execution time: {execution_time}')